# Save data to website

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import pandas as pd

In [4]:
data_path = '../data/'
venue_short = 'iclr2023'
date = time.strftime("%Y%m%d")
df = pd.read_csv(f'{data_path}{venue_short}_{date}.csv')

# get average rating for each paper from ratings column
df['ratings_avg'] = df['ratings'].apply(lambda x: np.mean(eval(x)))
df['ratings_std'] = df['ratings'].apply(lambda x: np.std(eval(x)))
df['confidence_avg'] = df['confidences'].apply(lambda x: np.mean(eval(x)))
df['confidence_std'] = df['confidences'].apply(lambda x: np.std(eval(x)))

/home/botu/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/botu/.local/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/botu/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/botu/.local/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


# Write all submissions

In [5]:
df['url'] = df['id'].apply(lambda x: 'https://openreview.net/forum?id=' + x)
df.head()

id                                              title  \
0  kRvZ2PcsxjJj                    Quantum reinforcement learning    
1   RUzSobdYy0V  Quantifying and Mitigating the Impact of Label...   
2   N3kGYG3ZcTi  Suppression helps: Lateral Inhibition-inspired...   
3   tmIiMPl4IPa                Factorized Fourier Neural Operators   
4   mhnHqRqcjYU  DFPC: Data flow driven pruning of coupled chan...   

                                            keywords          ratings  \
0  ['quantum reinforcement learning', 'multi-agen...     [1, 1, 1, 1]   
1                                                 []        [5, 6, 8]   
2  ['Lateral Inhibition', 'Convolutional Neural N...     [3, 5, 3, 1]   
3  ['fourier transform', 'fourier operators', 'pd...  [8, 6, 3, 8, 3]   
4      ['Pruning', 'Data Free', 'Model Compression']        [8, 6, 6]   

       confidences  withdraw             review_lengths  ratings_avg  \
0     [5, 5, 5, 5]         0          [45, 49, 25, 283]     1.000000   
1        [4, 3, 3]         0            [443, 274, 401]     6.333333   
2     [5, 5, 5, 5]         0       [333, 360, 362, 304]     3.000000   
3  [5, 4, 4, 2, 2]         0  [203, 142, 323, 520, 635]     5.600000   
4        [3, 2, 3]         0             [302, 90, 257]     6.666667   

   ratings_std  confidence_avg  confidence_std  \
0     0.000000        5.000000        0.000000   
1     1.247219        3.333333        0.471405   
2     1.414214        5.000000        0.000000   
3     2.244994        3.400000        1.200000   
4     0.942809        2.666667        0.471405   

                                            url  
0  https://openreview.net/forum?id=kRvZ2PcsxjJj  
1   https://openreview.net/forum?id=RUzSobdYy0V  
2   https://openreview.net/forum?id=N3kGYG3ZcTi  
3   https://openreview.net/forum?id=tmIiMPl4IPa  
4   https://openreview.net/forum?id=mhnHqRqcjYU

In [6]:
# if none, set to 0
df['ratings_avg'] = df['ratings_avg'].fillna(0)

# get papers with 0 ratings_avg and set ratings column as 'rating not available'ArithmeticError
df.loc[df['ratings_avg'] == 0, 'ratings'] = 'Ratings not available yet'
# df[df['ratings_avg'] == 0]['ratings'] = 'Rating not available yet'

In [18]:
# read template 
with open('../web/openreview_template_rebuttal.html', 'r') as f:
    html_temp = f.readlines()

# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1


# write data
for i, el in df.iterrows():
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{el['url']}'> {el['title']}</a></td>" \
           f"<td class='r'>{el['ratings_avg']:.2f}</td" \
           f"<td data-sort='{el['ratings_avg']}'>" \
           f"<td data-sort='{len(el['ratings'])}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r'>{', '.join(el['ratings'].strip('[]').split(','))}</td>" \
           f"</table></td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open(f'../submissions.html' ,'w') as f:
    f.write("".join(html_temp))